In [2]:
import torch
from torch import nn
# from d2l import torch as d2l

In [5]:
def corr2d(X,K):   #X是输入，，K是核矩阵
    #计算二维互相关运算 
    h,w = K.shape   #K 的行数和列数
    Y = torch.zeros((X.shape[0] - h+1,X.shape[1]-w+1))   #Y--输出的高度和宽度
    for i in range(Y.shape[0]):
        for j in range (Y.shape[1]):
            Y[i,j] = (X[i:i +h,j:j +w] * K).sum()   #X的第i行到第i+h行，第j列到第j+w列   再和K做点积，再求和
    return Y

In [6]:
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

In [7]:
#实现二维卷积层
class Conv2D(nn.Module): 
    def __init__(self,kernel_size):   #kernel.size是超参数，可以自己设置
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self,x):
        return corr2d(x,self.weight) + self.bias

In [8]:
#卷积层的简单应用，检测图像中不同颜色的边缘

X= torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [13]:
K = torch.tensor([[1.0,-1.0]])

In [14]:
Y = corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [15]:
#卷积核K只可以检测垂直边缘
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [19]:
#学习由x生成Y的卷积核

conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)  #输入通道为1，输出通道为1

X = X.reshape((1,1,6,8))  # 通道数为1，批量大小数为1,6X8
Y = Y.reshape((1,1,6,7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i+1) % 2 == 0:
        print(f'batch{i+1},loss{l.sum(): .3f}')





batch2,loss 0.868
batch4,loss 0.190
batch6,loss 0.050
batch8,loss 0.016
batch10,loss 0.006


In [20]:
#所学习的卷积核的权重张量
conv2d.weight.data.reshape((1,2))


tensor([[ 1.0031, -0.9883]])

In [21]:
#填充和步幅
#在所有侧边填充1个像素

def comp_conv2d(conv2d,X):
    X = X.reshape((1,1) + X.shape)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

conv2d = nn.Conv2d(1,1,kernel_size = 3,padding=1)   #填充为1，上下左右各填充一行#
X = torch.rand(size=(8,8))
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [22]:
#填充不同的高度和宽度
conv2d = nn.Conv2d(1,1,kernel_size = (5,3),padding=(2,1))  #填充，上下加2，左右加1
comp_conv2d(conv2d,X).shape

torch.Size([8, 8])

In [24]:
#将高度和宽度的步幅设置为2
conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)# stride=2步幅为2
comp_conv2d(conv2d,X).shape

torch.Size([4, 4])

In [26]:
#一个稍微复杂的例子
conv2d = nn.Conv2d(1,1,kernel_size=(3,5),padding=(0,1),stride=(3,4))
comp_conv2d(conv2d,X).shape

torch.Size([2, 2])